In [ ]:
import sys
sys.path.append('..') 

from keithleyAPI import Communications
from tests_ import sensing_test,stability_test
import utils

In [ ]:
import plotly.express as px 
import pandas as pd
import seaborn as sns
import time
import matplotlib.pyplot as plt
import os
import numpy as np

smu = Communications("TCPIP0::169.254.181.21::1225::SOCKET")
smu.connect()
smu._instrument_object.write_termination = "\0"
smu._instrument_object.read_termination = "\0"

In [ ]:
DUT = 'd030824-II' # device under test
couple = 'r3c1'
VDL = 'CH1'
VDR = 'CH3'
CMN = 'CH2'

In [ ]:
smu.query("BC") # clears all readings from buffer

# Diode connection Test

### To run a cell and go to the next one: Shift + enter
#### Run the following cell until it prints out "Device correctly stabilized"
Be sure to change the device under test on the last line

In [ ]:
diode_df = []
mean_diff = []
TOT = 'DiodeConnected_StabilityforIonSensing-Na' #type of test

### Run Stability Check before sensing test

In [ ]:
 diode_df, mean_diff = stability_test('CH1', 'CH3', 'CH2', smu,diode_df,mean_diff,'sensing',couple,DUT,TOT,max_steps = 10,resting_time=10)

In [ ]:
utils.save_xls(diode_df, DUT+couple,TOT,additional_comment= '80sweeps', mode = 2)

#### Run the following cell to plot the max values in time for the last sweeps

In [ ]:
utils.plot_max_values(diode_df,['baseline'],couple,1,DUT,TOT,3,r'C:\Users\Desi\Desktop\TesiStanford\images-results')

#### Plot Diode Connection

In [ ]:
# plot DiodeConnection
fig, ax1 = plt.subplots(figsize=(8, 8))
ax2 = ax1.twinx()

ax1.plot(diode_df[-1]['IDL'], diode_df[-1]['VDL'], color = 'blue', label = 'VDL')
ax1.set_ylabel('VdLeft [V]')
ax2.plot(diode_df[-1]['IDL'], diode_df[-1]['VDR'], color = 'red', label = 'VDR')
ax2.set_ylabel('VdRight [V]')
plt.grid()
ax1.set_xlabel('Id [A]')
plt.title('Diode Connected - Single Sweep')
ax1.legend()
ax2.legend(loc = 1)


# Sensing Test

### Run the following two cells only one time!

In [ ]:
meandiff = []
stddiff = []
diode_dict_list = {} # dictionary where to store the list with the 20 sweeps of all the concentrations
diode_df_dict = {} # dictionary where to store the df with the 20 sweeps of all the concentrations
mean_std = [] #list where to store the mean and std of the òast 5 values of the last Nvalidsteps
mean_std_L = []
mean_std_R = []
baseline = 0

In [ ]:
k = 0 
conc = ['baseline','1pM','10pM','100pM','1nM','10nM','100nM','1uM']
#conc = ['1mM','2mM','4mM','8mM','16mM','32mM']

In [ ]:
conc = ['10mM','20mM','40mM','80mM','160mM','16mMK','32mMK']
#conc = ['1mM','2mM','4mM','8mM','10mMNa','16mM','20mMNa','32mM']
#conc = ['baseline','1pM_CT','1pM_MT','10pM_CT','10pM_MT','100pM_CT','100pM_MT','1nM_CT','1nM_MT','10nM_CT','10nM_MT','100nM_CT','100nM_MT','1uM_CT','1uM_MT']
#conc = ['baseline','1pM','10pM','100pM','1nM','10nM','100nM','1uM','1nM-MT','100nm_MT']

### Run all the following cells until 'END' every time you change concentration

In [ ]:
TOT = 'DiodeConnected_SensingTest-Na' #Type Of Test
k, diode_df_dict, diode_dict_list, mean_std, mean_std_L, mean_std_R, folder, baseline = sensing_test('CH1', 'CH3', 'CH2', smu,k, conc, diode_df_dict, diode_dict_list, mean_std, mean_std_L, mean_std_R,DUT, TOT, couple, baseline)

In [ ]:
# printing the mean and the std of the last 5 values of the 20 sweeps. Each [] represents a concentration

# plotting change of MAx values in time for different concentrations
TOT = 'changeMaxDiffvaluesintime'
utils.plot_max_values(diode_df_dict,conc,couple,1,DUT,TOT, mode = 1, folder)

# plotting change of Mx values in time for different concentrations for left and right electrodes
TOT = 'changeMaxDiffvaluesintime-LR'
utils.plot_max_values(diode_df_dict,conc,couple,1,DUT,TOT, mode = 2, folder)

# plotting mean and std of the diff for different concentrations
utils.plot_mean_std(len(mean_std), mean_std_L,mean_std_R, mean_std, conc, couple, folder)



#### Interactive plot

In [ ]:
diff_in_time = pd.concat(diode_df_dict.values(), ignore_index=False)
numberoftests = len(diode_df_dict)
diff_in_time_grouped = diff_in_time.groupby(diff_in_time.index)
threshold = diff_in_time.index.max()

diff_in_time_max = [group for name, group in diff_in_time_grouped if name >= threshold]
fig = px.scatter(diff_in_time_max[0]-diff_in_time_max[0]['VDL'].iloc[0], x=range(len(diff_in_time_max[0])), y='VDL')
fig.data[0].name="VDL"
fig.update_traces(showlegend=True)
 
fig.add_scatter(x=np.array(range(len(diff_in_time_max[0]))), y=diff_in_time_max[0]['VDR']-diff_in_time_max[0]['VDR'].iloc[0], mode='markers',name = "VDR")
fig.add_scatter(x=np.array(range(len(diff_in_time_max[0]))), y=(diff_in_time_max[0]['VDL']-diff_in_time_max[0]['VDR'])-(diff_in_time_max[0]['VDL']-diff_in_time_max[0]['VDR']).iloc[0], mode='markers',name = "diff")
 
fig.show()

In [ ]:
# std and mean only for the diff

In [ ]:
plt.scatter(conc[:k], [i[0]*1000 for i in mean_std] )
plt.errorbar(conc[:k], [i[0]*1000 for i in mean_std], yerr=[i[1]*1000 for i in mean_std], label='std')

plt.xlabel('Concentration')
plt.ylabel('DeltaV [mV]')
plt.title('Mean and Std DeltaV dor different concentrartions (last 5 values of the last 6 steps)')
plt.legend()
plt.grid()


### End